# TF-IDF with KNN

In [1]:
# import
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# load dataset
df = pd.read_csv('tf-idf_halodocnew_1.csv')
print(df.shape)

# check dataset
df.head()

(9979, 16)


,url,title,title_count,content,content_count,title_tokens,content_tokens,title_tokens_stopwords,title_tokens_stopwords_count,content_tokens_stopwords,content_tokens_stopwords_count,title_tokens_stopwords_stemmed,title_tokens_stopwords_stemmed_count,content_tokens_stopwords_stemmed,content_tokens_stopwords_stemmed_count,document_id
0,https://www.halodoc.com/ibu-perhatikan-5-hal-i...,"ibu, perhatikan 5 hal ini saat memilih car se...",11,"bagi orang tua yang memiliki kendaraan, memili...",575,"['ibu', ',', 'perhatikan', '5', 'hal', 'ini', ...","['bagi', 'orang', 'tua', 'yang', 'memiliki', '...","[',', 'perhatikan', '5', 'memilih', 'car', 'se...",7,"['orang', 'tua', 'memiliki', 'kendaraan', ',',...",339,"['', 'perhati', '5', 'pilih', 'car', 'seat', '...",7,"['orang', 'tua', 'milik', 'kendara', '', 'memi...",339,0
1,https://www.halodoc.com/catat-ini-6-tips-menyi...,"catat, ini 6 tips menyimpan beras agar tidak ...",10,sebagai makanan pokok beras harus disimpan den...,617,"['catat', ',', 'ini', '6', 'tips', 'menyimpan'...","['sebagai', 'makanan', 'pokok', 'beras', 'haru...","['catat', ',', '6', 'tips', 'menyimpan', 'bera...",7,"['makanan', 'pokok', 'beras', 'disimpan', 'kua...",347,"['catat', '', '6', 'tips', 'simpan', 'beras', ...",7,"['makan', 'pokok', 'beras', 'simpan', 'kualita...",347,1
2,https://www.halodoc.com/waspada-ini-6-bahaya-a...,"waspada, ini 6 bahaya asap rokok bagi kesehat...",10,merokok tidak hanya berbahaya untuk dirimu sen...,708,"['waspada', ',', 'ini', '6', 'bahaya', 'asap',...","['merokok', 'tidak', 'hanya', 'berbahaya', 'un...","['waspada', ',', '6', 'bahaya', 'asap', 'rokok...",8,"['merokok', 'berbahaya', 'dirimu', 'orang-oran...",438,"['waspada', '', '6', 'bahaya', 'asap', 'rokok'...",8,"['rokok', 'bahaya', 'diri', 'orang', 'sekitar'...",438,2
3,https://www.halodoc.com/sering-terkena-polusi-...,"sering terkena polusi, ketahui berbagi cara b...",9,"sehari-hari, manusia terpapar polusi dalam jum...",569,"['sering', 'terkena', 'polusi', ',', 'ketahui'...","['sehari-hari', ',', 'manusia', 'terpapar', 'p...","['terkena', 'polusi', ',', 'ketahui', 'berbagi...",7,"['sehari-hari', ',', 'manusia', 'terpapar', 'p...",342,"['kena', 'polusi', '', 'tahu', 'bagi', 'sih', ...",7,"['hari', '', 'manusia', 'papar', 'polusi', 'as...",342,3
4,https://www.halodoc.com/ini-risiko-dan-manfaat...,ini risiko dan manfaat implan payudara yang p...,9,implan payudara adalah prosedur bedah kosmetik...,613,"['ini', 'risiko', 'dan', 'manfaat', 'implan', ...","['implan', 'payudara', 'adalah', 'prosedur', '...","['risiko', 'manfaat', 'implan', 'payudara']",4,"['implan', 'payudara', 'prosedur', 'bedah', 'k...",362,"['risiko', 'manfaat', 'implan', 'payudara']",4,"['implan', 'payudara', 'prosedur', 'bedah', 'k...",362,4


In [2]:
import ast

df['title_tokens'] = df['title_tokens'].apply(ast.literal_eval)
df['content_tokens'] = df['content_tokens'].apply(ast.literal_eval)
df['title_tokens_stopwords'] = df['title_tokens_stopwords'].apply(ast.literal_eval)
df['content_tokens_stopwords'] = df['content_tokens_stopwords'].apply(ast.literal_eval)
df['content_tokens_stopwords_stemmed'] = df['content_tokens_stopwords_stemmed'].apply(ast.literal_eval)
df['title_tokens_stopwords_stemmed'] = df['title_tokens_stopwords_stemmed'].apply(ast.literal_eval)


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# # defining the TF-IDF
# tfidf_configs = {
#     'lowercase': True,
#     'analyzer': 'word',
#     'stop_words': 'english',
#     'binary': True,
#     'max_df': 0.9,
#     'max_features': 10_000
# }
# defining the number of documents to retrieve
retriever_configs = {
    'n_neighbors': 10,
    'metric': 'cosine'
}

# defining our pipeline
embedding = TfidfVectorizer()
retriever = NearestNeighbors(**retriever_configs)

In [4]:
df['document_id'] = range(len(df))

In [5]:
X = embedding.fit_transform(df['content_tokens_stopwords_stemmed'].apply(lambda tokens: ' '.join(tokens)))
retriever.fit(X, df['document_id'])

NearestNeighbors(metric='cosine', n_neighbors=10)

In [23]:
# question/query as input
question = 'bagaimana covid-19 sekarang?'

In [24]:
def transform_text(vectorizer, text):
    print('Text:', text)
    vector = vectorizer.transform([text])
    vector = vectorizer.inverse_transform(vector)
    print('Vect:', vector)

# vectorize the question
transform_text(embedding, question)

Text: bagaimana covid-19 sekarang?
Vect: [array(['sekarang', 'covid', 'bagaimana', '19'], dtype='<U72')]


In [25]:
# Stemming (using Sastrawi) => for question/query
factory = StemmerFactory()
stemmer = factory.create_stemmer()
stop_words = set(stopwords.words('indonesian'))

# stopwords removal and stemming question
question = [stemmer.stem(word) for word in word_tokenize(question) if word not in stop_words]
question = ' '.join(question)


# predict the most similar document
X = embedding.transform([question])
document_id = retriever.kneighbors(X, return_distance=False)[0][0]
selected = df.iloc[document_id]['content']

selected

'kondisipandemi covid-19kini sudah terlihat membaik di beberapa negara. bahkan di indonesia, kasus aktif covid-19 terus menunjukkan angka penurunan. menurut juru bicara vaksinasi covid-19, dr. siti nadia tarmizi m. epid, kasus aktif covid-19 mengalami penurunan sejak februari lalu. hal ini membuat presiden indonesia, joko widodo optimis dapat menghentikan pandemi covid-19 dalam waktu dekat. hal serupa juga dirasakan oleh direktur jenderal organisasi kesehatan dunia (who), tedros adhanom, yang mengatakan bahwa akhir pandemi sudah di depan mata. benarkah demikian? yuk, simak ulasannya berikut ini!  benarkah pandemi covid-19 segera berakhir? presiden joko widodo menyampaikan bahwa pemerintah memiliki kesempatan untuk menyatakan pandemi covid-19 di indonesia akan berakhir. hal ini terjadi karena kasus aktif covid-19 di indonesia mengalami angka penurunan yang signifikan. hal ini disampaikan oleh juru bicara vaksinasi covi-19, dr. siti nadia tarmizi, bahwa angka aktif covid-19 mengalami pen

In [26]:
# vectorize the document
transform_text(embedding, selected)

Text: kondisipandemi covid-19kini sudah terlihat membaik di beberapa negara. bahkan di indonesia, kasus aktif covid-19 terus menunjukkan angka penurunan. menurut juru bicara vaksinasi covid-19, dr. siti nadia tarmizi m. epid, kasus aktif covid-19 mengalami penurunan sejak februari lalu. hal ini membuat presiden indonesia, joko widodo optimis dapat menghentikan pandemi covid-19 dalam waktu dekat. hal serupa juga dirasakan oleh direktur jenderal organisasi kesehatan dunia (who), tedros adhanom, yang mengatakan bahwa akhir pandemi sudah di depan mata. benarkah demikian? yuk, simak ulasannya berikut ini!  benarkah pandemi covid-19 segera berakhir? presiden joko widodo menyampaikan bahwa pemerintah memiliki kesempatan untuk menyatakan pandemi covid-19 di indonesia akan berakhir. hal ini terjadi karena kasus aktif covid-19 di indonesia mengalami angka penurunan yang signifikan. hal ini disampaikan oleh juru bicara vaksinasi covi-19, dr. siti nadia tarmizi, bahwa angka aktif covid-19 mengalam